In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

In [3]:
from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet
from ordinaloss.utils.loss_utils import CSCELoss, create_ordinal_cost_matrix
from ordinaloss.engine.model_engine import OrdinalEngine


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

loaded ordinaloss.utils.pretrained_models


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

False

In [5]:
train_transform = create_transform_pipeline(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3) #These are hyperparameters. 
test_transform = create_transform_pipeline() #Without color jitter for test pipeline

train_dataset = ImageFolder("../datasets/kneeKL224/train/", transform=train_transform)
val_dataset = ImageFolder("../datasets/kneeKL224/val/", transform = test_transform)
test_dataset = ImageFolder("../datasets/kneeKL224/test/", transform = test_transform)
auto_test_dataset = ImageFolder("../datasets/kneeKL224/auto_test/", transform = test_transform)

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size = 128, shuffle=False)

In [6]:
model = classification_model_resnet("resnet18", num_classes = 5) #Could also be resnet34, resnet50, resnet101, resnet152, 
# model = classification_model_densenet("densenet121", num_classes = 5) #Could also be densenet169, densenet201

In [8]:
engine = OrdinalEngine(model, 
                       loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
                       device = device, #change to gpu, let me know if it doesnt work on GPU.
                       use_lr_scheduler=True, lr_decay_epoch=5, lr_decay_factor= 0.8,
                       optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4, momentum = 0.9
                       )

In [9]:
engine.train(test_loader, test_loader, n_epochs = 10)

  0%|          | 0/13 [00:00<?, ?it/s]

LR is set to 0.001
LR is set to 0.001


100%|██████████| 13/13 [00:59<00:00,  4.59s/it, accuracy=0.386, batch_size=1656, loss=221, mae=1.25]   


{'loss': 221.22698786379635, 'accuracy': 0.3858695652173913, 'mae': 1.2457729434045617}


  0%|          | 0/13 [00:00<?, ?it/s]

LR is set to 0.0009000000000000001
LR is set to 0.0009000000000000001


 62%|██████▏   | 8/13 [01:56<01:12, 14.58s/it, accuracy=0.549, batch_size=1024, loss=25.1, mae=0.6] 


KeyboardInterrupt: 